<a href="https://colab.research.google.com/github/AnitaKirkovska/Bio_Project_Portfolio/blob/master/model_training_only.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This version was used for training purposes only. We wanted to analyze how different time points from our data collection were chainging the training results.

**Training results we got from this model:**

*   81% for time1
*   60% for time2
*   64% for time3
*   58% for time2 and time3
*   62% for all 3 times



# Make the Model

In [0]:
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, Conv1D, Conv2D, BatchNormalization, MaxPool1D, Concatenate
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import BatchNormalization

"""
# build the imag CNN
vgg = VGG19(
    weights = 'imagenet',
    include_top = False,
    input_shape = (224, 224, 3)
)
img_base = vgg.output
img_base = Flatten()(img_base)
img_base = Dense(4096)(img_base)
"""

# Build the hyperspec 1DCNN

hyperspec_input = Input(shape=(2554, 1))
hyperspec_base = Conv1D(filters=20, kernel_size=24, strides=1, padding='valid', activation='tanh')(hyperspec_input)
hyperspec_base = BatchNormalization()(hyperspec_base)
hyperspec_base = MaxPool1D(pool_size=5, strides=None)(hyperspec_base)
hyperspec_base = Flatten()(hyperspec_base)


hyperspec_base = Flatten()(hyperspec_base)
#hyperspec_base = Dense(512)(hyperspec_base)

"""
# Combine the feature maps
combined = Concatenate()([img_base, hyperspec_base])
"""
z = Dense(100, activation='tanh')(hyperspec_base)
z = Dense(1, activation = 'sigmoid')(z)

# Create the full model
model = Model(hyperspec_input, z)
#plot_model(model, show_shapes=True, show_layer_names=True)

In [0]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 2554, 1)]         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2531, 20)          500       
_________________________________________________________________
batch_normalization_2 (Batch (None, 2531, 20)          80        
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 506, 20)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 10120)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 10120)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               1012

# Upload Data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving Data.zip to Data.zip


## Unzip Data

In [0]:
!unzip Data.zip

Archive:  Data.zip
   creating: Data/
   creating: Data/Week_3/
   creating: Data/Week_3/Hyperspecs/
  inflating: Data/Week_3/Hyperspecs/C_1_DEC_ON_6N_Splice17_11-53-52-056.txt  
   creating: __MACOSX/
   creating: __MACOSX/Data/
   creating: __MACOSX/Data/Week_3/
   creating: __MACOSX/Data/Week_3/Hyperspecs/
  inflating: __MACOSX/Data/Week_3/Hyperspecs/._C_1_DEC_ON_6N_Splice17_11-53-52-056.txt  
  inflating: Data/Week_3/Hyperspecs/N_2_HIR_IL_4N_Splice17_12-10-43-381.txt  
  inflating: __MACOSX/Data/Week_3/Hyperspecs/._N_2_HIR_IL_4N_Splice17_12-10-43-381.txt  
  inflating: Data/Week_3/Hyperspecs/C_1_DEC_NC_4N_Splice17_11-44-14-875.txt  
  inflating: __MACOSX/Data/Week_3/Hyperspecs/._C_1_DEC_NC_4N_Splice17_11-44-14-875.txt  
  inflating: Data/Week_3/Hyperspecs/N_1_DEC_CT_2N_Splice17_11-34-47-961.txt  
  inflating: __MACOSX/Data/Week_3/Hyperspecs/._N_1_DEC_CT_2N_Splice17_11-34-47-961.txt  
  inflating: Data/Week_3/Hyperspecs/C_1_HIR_WI_4N_Splice17_11-41-40-538.txt  
  inflating: __MACOSX

# Seperate Data into Control and Nitrogen

In [0]:
import os
import glob

# {'control' : set(ctrl_plant_name1, ctrl_plant_name2, ...)
#	'nitrogen' : set(nitrogen_plant_name1, nitrogen_plant_name2, ...)}
plants = {'control': set(), 'nitrogen':set()}

# {plant_name1 : [plant_name1_hyperspec1, plant_name1_hyperspec2, ...],
#	plant_name2 : [plant_name2_hyperspec1, plant_name2_hyperspec2, ...]}
hyperspecs = {}

# {plant_name1 : [plant_name1_img1, plant_name1_img2, ...],
#	plant_name2 : [plant_name2_img1, plant_name2_img2, ...]}
imgs = {}

data_path = './Data'
data_path = os.path.abspath(data_path)
weeks = glob.glob(os.path.join(data_path, '*'))

# Returns the class of the plant, and the formatted plant name
def get_plant(filepath):
	filename = os.path.basename(filepath)
	split = filename.split('_')
	plant_name = split[0] + '_' + split[1] + '_' + split[2] + '_' + split[3] + '_' + split[4]
	plant_name = plant_name.upper()

	if (split[0] == 'c' or split[0] == 'C'):
		category = 'control'
	elif (split[0] == 'n' or split[0] == 'N'):
		category = 'nitrogen'
	else:
		print('Unknown Class:', filename)

	return [category, plant_name]


for week in weeks:
	hyperspec_files = glob.glob(os.path.join(week, 'Hyperspecs', '*'))
	img_files = glob.glob(os.path.join(week, 'Images', '*'))

	for hyperspec_file in hyperspec_files:
		category, plant_name = get_plant(hyperspec_file)
		plants[category].add(plant_name)

		h = hyperspecs.get(plant_name, [])
		h.append(hyperspec_file)
		hyperspecs[plant_name] = h

	for img_file in img_files:
		category, plant_name = get_plant(img_file)
		plants[category].add(plant_name)

		i = imgs.get(plant_name, [])
		i.append(img_file)
		imgs[plant_name] = i

print(hyperspecs)

print('Num Control', len(plants['control']))
print('Num Nitrogen', len(plants['nitrogen']))

{'N_2_DIV_WI_4N': ['/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-49-537.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-47-246.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-18-08-619.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-18-10-990.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-38-342.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-18-02-086.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-53-182.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-35-370.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-55-898.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-18-08-021.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-18-05-053.txt', '/content/Data/Week_3/Hyperspecs/N_2_DIV_WI_4N_Splice17_12-17-36-817.txt'], 'C_1_DEC_CT_2N': ['/content/Data/Week_3/Hyperspecs/C_1_DEC_CT_2N_Splice17_11-46-

# Data Generator

In [0]:
import os
import glob
import numpy as np
import tensorflow.keras as keras

class data_gen(keras.utils.Sequence):
	def get_data(self):
		# {'control' : set(ctrl_plant_name1, ctrl_plant_name2, ...)
		#	'nitrogen' : set(nitrogen_plant_name1, nitrogen_plant_name2, ...)}
		plants = {'control': [], 'nitrogen': []}

		# {plant_name1 : [plant_name1_hyperspec1, plant_name1_hyperspec2, ...],
		#	plant_name2 : [plant_name2_hyperspec1, plant_name2_hyperspec2, ...]}
		hyperspecs = {}

		data_path = './Data'
		data_path = os.path.abspath(data_path)
		weeks = glob.glob(os.path.join(data_path, '*'))

		for week in weeks:
			hyperspec_files = glob.glob(os.path.join(week, 'Hyperspecs', '*'))
			img_files = glob.glob(os.path.join(week, 'Images', '*'))

			for hyperspec_file in hyperspec_files:
				filename = os.path.basename(hyperspec_file)
				split = filename.split('_')
				plant_name = split[0] + '_' + split[1] + '_' + split[2] + '_' + split[3] + '_' + split[4]
				plant_name = plant_name.upper()

				if (split[0] == 'c' or split[0] == 'C'):
					category = 'control'
				elif (split[0] == 'n' or split[0] == 'N'):
					category = 'nitrogen'
				else:
					print('Unknown Class:', filename)

				plants[category].append(plant_name)

				h = hyperspecs.get(plant_name, [])
				h.append(hyperspec_file)
				hyperspecs[plant_name] = h

		return plants, hyperspecs


	def __init__(self, batch_size, dim):
		self.batch_size = int(batch_size // 2 * 2)
		self.dim = dim
		self.plants, self.hyperspecs = self.get_data()
		self.epoch_data = {}
		self.on_epoch_end()


	def __len__(self):
		num_plants = len(plants['control']) + len(plants['nitrogen'])
		return int(np.ceil(num_plants / self.batch_size))


	def on_epoch_end(self):
		self.epoch_data['control'] = np.random.choice(self.plants['control'], len(self.plants['control']), replace=False)
		self.epoch_data['nitrogen'] = np.random.choice(self.plants['nitrogen'], len(self.plants['nitrogen']), replace=False)


	def __getitem__(self, index):
		start = int(index * self.batch_size / 2)
		end = int((index + 1) * self.batch_size / 2)
		control_plants = self.epoch_data['control'][start:end].tolist()
		nitrogen_plants = self.epoch_data['nitrogen'][start:end].tolist()

		num_plants = len(control_plants) + len(nitrogen_plants)

		X = np.empty((num_plants, self.dim, 1))
		y = np.empty((num_plants, 1))

		for i, plant in enumerate(control_plants):
			X[i] = self.read_hyperspec(plant)
			y[i] = 0
		for j, plant in enumerate(nitrogen_plants):
			X[i+j+1] = self.read_hyperspec(plant)
			y[i+j+1] = 1

		return X, y


	def read_hyperspec(self, plant):
		filepath = np.random.choice(self.hyperspecs[plant], 1)[0]

		with open(filepath, 'r') as f:
			lines = [l.strip('\n\r') for l in f]

		found_delimiter = False
		for i, line in enumerate(lines):
			if line == '>>>>>Begin Spectral Data<<<<<':
				found_delimiter = True
				break

		if not found_delimiter:
			print('Hyperspec File NOT Formatted as Expected')
			print(file)
			quit()

		X = []
		data = lines[i+1:]
		for datum in data:
			wavelen, intense = datum.split('\t')
			X.append(float(intense))

		X = np.array(X).reshape((self.dim, 1))
		return X

# Train Model

In [0]:
from tensorflow.keras import optimizers
train_gen = data_gen(8, 2554)
opt = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9)

model.compile(loss='mean_absolute_error', optimizer=opt, metrics=['accuracy'])
model.fit_generator(generator=train_gen, epochs=500)

Epoch 1/500
8/8 [==============================] - 0s 20ms/step - loss: 0.3752 - accuracy: 0.6250
Epoch 2/500
8/8 [==============================] - 0s 21ms/step - loss: 0.3906 - accuracy: 0.6094
Epoch 3/500
8/8 [==============================] - 0s 21ms/step - loss: 0.2837 - accuracy: 0.7188
Epoch 4/500
8/8 [==============================] - 0s 21ms/step - loss: 0.3130 - accuracy: 0.6875
Epoch 5/500
8/8 [==============================] - 0s 21ms/step - loss: 0.3287 - accuracy: 0.6719
Epoch 6/500
8/8 [==============================] - 0s 22ms/step - loss: 0.3595 - accuracy: 0.6406
Epoch 7/500
8/8 [==============================] - 0s 22ms/step - loss: 0.3453 - accuracy: 0.6562
Epoch 8/500
8/8 [==============================] - 0s 21ms/step - loss: 0.3449 - accuracy: 0.6562
Epoch 9/500
8/8 [==============================] - 0s 22ms/step - loss: 0.3754 - accuracy: 0.6250
Epoch 10/500
8/8 [==============================] - 0s 21ms/step - loss: 0.3289 - accuracy: 0.6719
Epoch 11/500
8/8 [=